Gradient boost

Короч суть в построении деревьев каждое из которых строиться на основании ошибок предыдущих деревьев. Предсказания делаются путем прогона экземпляра по всем деревьям 

Глубина каждого дерева ограничивается, глубина леса может тоже

Regression

Loss function - SSR/2, на два делят вроде чтобы производную легче вычислять было

Сначала пытается создать лист , а не дерево для предсказания (изначальный лист)

Первый guess будет среднее по Y

Затем GB строит дерево 

Сначала определяем ошибку для каждого экземпляра(разницу между target и predicted) - pseudo residual 

Затем строим дерево пытаясь предсказать не Y, а pseudo residuals (residuals)

Теперь обновляем колонку pseudo residuals

Для этого прогоняем экземпляр по новому дереву и получаем результат предсказанный

Потом берём предсказание изначального листа (среднее по y) и прибавляем к нему значение получившееся после прогона экземпляра по дереву умноженное на learning rate (для борьбы с переобучением нужно) 

Затем из реального Y для этого экземпляра вычитаем сумму

То есть
residual для экземпляра = target - (pred из изначального листа + l_rate* pred из дерева)

Так по всему датасету прогоняем и вычисляем новые residuals

Строим ещё одно дерево которое пытается предсказать новые residuals

Теперь вычисляем новые residuals прогоняя экземпляры уже через изначальный лист + первое дерево + второе дерево (результат каждого дерево умножаем на learning rate)

Каждый раз pseudo residuals будут уменьшаться (значит мы в верном направлении идём)

Короч добавляем таким же образом новые деревья пока либо не достигнем максимального колв-ва деревьев которое выбрали, либо когда добавление новых деревьев перестанет значительно уменьшать значения residuals

Параметры - 
Макс кол-во листьев в дереве обычно от 8 до 32 ставят
Макс кол-во деревьев 


Classifcation

Также создаём изначальный лист (initial prediction), в регрессии это было среднее по Y, в классификации это log(odds), те у 4х челов Y равен классу А, у 2х классу B - log(4/2)

Затем с помощью logistic функции (сигмовидной) конвертим значение в вероятность

В нашем случае получили 0.7 

То есть наш лист для всех выдает вероятность 0.7, что экземпляр относится к классу А

Выбираем threshold (0.5 обычно)

Те если predicted probability >= 0.5, то класс А, если нет, то класс Б

Тк наш лист выдает 0.7, то мы все экземпляры относим к классу А

Затем вычисляем residuals (чтобы понять насколько плохо наш лист перформит)

Берём реальные Y у экземпляров (если экземпляр к классу А относится то Y = 1 у него, если к классу Б, то 0 

И для каждого экземпляра residual будет равен observed(1 или 0 в зависимости от класса) минус predicted (0.7)

Теперь строим дерево для предугадывания residuals

Построили дерево ( в листах соотвно получили 1 или больше residuals (ТК мы их угадывать учились))

Теперь должны трансформировать эти предсказания в листах дерева, ТК дерево было обучено на residuals, которые были выведены из вероятностей (от 1 или 0 отнимали 0.7), а предсказания имеют природу log(odds). То есть мы не можем сложить все значения в каком-то листе и найти среднее как в случае с регрессией

 Формула для трансформации
Сумма residuals (то что в листе дерева) деленная на сумму предыдущая вероятность (0.7 в нашем случае) умноженная на 1- предыдущая вероятность 

Сделали трансформацию и получили конечные значения которые будет выдавать дерево когда через него прогонят экземпляр 

То есть сначала у нас в листьях деревьев было 1 или больше residuals на которых это дерево обучили, а после трансформации в каждом листе по новому значению (аналог среднего в деревьях регрессии)

Теперь вычисляем новое предсказание - берем изначальный лист (он log(4/2) предсказывает прибавляем learning rate умноженный на предсказание построенного дерева и конвертим его с помощью logistic function

И так вычисляем новую predicted probability для каждого экземпляра

И вычисляем новые residuals (obzerved probability (1 или 0) минус predicted probability (уже не 0.7 для каждого, а для каждого своя, ТК эту новую probability мы вычисляем с помощью изначального листа и дерева через которые прогоняем экземпляры)

Строим новое дерево , также трансформируем значения в его листьях

И добавляем это дерево в нашу модель

Теперь как делаем предсказания с нашей моделью 

Берём экземпляр по которому нужно У определить

И прогоняем его через модель (изначальный лист и все деревья построенные), полученную log(odds) превращаем в probability ( с пом logistic function), сравниваем с трешхолдом и даём ответ